In [1]:
import os
import json
import numpy as np
import robotic as ry
from tqdm import tqdm
from lookup import features, contraint_types
from komo_generators import generate_random_komo, generate_random_pick, generate_random_pivot
from random_kitchen_generator import generate_random_kitchen, load_single_from_type
from MuJoCo2Rai.io import load_kitchen

Mujoco2Rai package has been imported!


In [2]:
C = load_kitchen()
C.addFile(ry.raiPath("scenarios/pandaOmnibase.g"))
C.view(True)

features["qItself"][1] = len(C.getJointState())

===================== ../kitchen_dataset/RUSTIC_ONE_WALL_SMALL.xml
| mujoco {'model': 'base'}
|   compiler {'angle': 'radian', 'meshdir': 'meshes/'}
|   option {'impratio': '20', 'density': '1.2', 'viscosity': '2e-05', 'cone': 'elliptic'}
|   size {'njmax': '5000', 'nconmax': '5000'}
|   visual {}
|     map {'znear': '0.001'}
|   asset {}
|     texture {'type': 'skybox', 'builtin': 'flat', 'rgb1': '1 1 1', 'rgb2': '1 1 1', 'width': '256', 'height': '1536'}
../textures/wood/wood_planks_wall.png
|     texture {'type': '2d', 'name': 'wall_room_wall', 'file': '../kitchen_dataset/../textures/wood/wood_planks_wall.png'}
../textures/flat/light_gray.png
|     texture {'type': '2d', 'name': 'wall_backing_room_wall', 'file': '../kitchen_dataset/../textures/flat/light_gray.png'}
../textures/wood/wood_planks_wall.png
|     texture {'type': '2d', 'name': 'wall_left_room_wall', 'file': '../kitchen_dataset/../textures/wood/wood_planks_wall.png'}
../textures/flat/light_gray.png
|     texture {'type': 

## Generate KOMO texts

In [3]:
frame_names = C.getFrameNames()

# Special Features (for random manipulation KOMOs)
end_effector_frames = ["r_gripper", "l_gripper"]
mobile_base_frames = ["omnibase"]
# object_frames = ["kebabs_8", "can_6", "pizza_cutter_2", "bell_pepper_0", "kebabs_13"]  # Frames that are not attached to anything that can be manipulated with basic pick, place, push, etc.
joint_frames = [fn for fn in frame_names if fn.endswith("hinge") or fn.endswith("_slidejoint")]
jointed_frames = [fn for fn in frame_names if fn.endswith("_handle_handle_visual")]
world_attach_frames = ["omnibase_world"]  # Like table in manipulation.py, some sort of world frame that tells us the world rotation and for us to place stuff after a mode switch.

for f in frame_names:
    print(f)
print(len(joint_frames))
print(len(jointed_frames))

base
wall_room_g0_vis
wall_backing_room_g0_vis
wall_left_room_g0_vis
wall_left_backing_room_g0_vis
wall_right_room_g0_vis
wall_right_backing_room_g0_vis
floor_room_g0_vis
floor_backing_room_g0_vis
outlet_room_g0
outlet_2_room_g0
light_switch_room_g0
light_switch_2_room_g0
sink_main_group_g0
sink_main_group_spout_joint_origin
sink_main_group_spout_joint
sink_main_group_g10
sink_main_group_handle_joint_origin
sink_main_group_handle_joint
sink_main_group_handle_temp_joint_origin
sink_main_group_handle_temp_joint
sink_main_group_g15
counter_main_main_group_base_right_visual
counter_main_main_group_base_left_visual
counter_main_main_group_base_back_visual
counter_main_main_group_base_front_visual
counter_main_main_group_top_left_visual
counter_main_main_group_top_right_visual
counter_main_main_group_top_front_visual
counter_main_main_group_top_back_visual
stove_main_group_g0
stove_main_group_g1
stove_main_group_knob_rear_right_joint_origin
stove_main_group_knob_rear_right_joint
stove_main_g

In [4]:
# komo_generator = lambda: generate_random_komo(frame_names, features, prob_mode_switch=0)
# komo_generator = lambda: generate_random_pick(end_effector_frames,
#                                               object_frames,
#                                               world_attach_frames,
#                                               frame_names,
#                                               drop_prob=0,
#                                               specific_orientation_prob=0,
#                                               hard_grasp_prob=.1,
#                                               specific_place_prob=0,
#                                               place_prob=1,
#                                               endeff_pass_prob=1,
#                                               repeat_pass_prob=1,
#                                               accumulated_collisions=False)
# komo_generator = lambda: generate_random_pivot(end_effector_frames, jointed_frames, joint_frames, slices=8)
def komo_generator():
    # if np.random.random() < .5:
    if np.random.random() < 1.:
        text = generate_random_pivot(end_effector_frames, jointed_frames, joint_frames, slices=1, max_back_and_forth_phases=3)
    else:
        text = generate_random_pick(end_effector_frames,
                                    object_frames,
                                    world_attach_frames,
                                    frame_names,
                                    drop_prob=0,
                                    specific_orientation_prob=.3,
                                    specific_place_prob=.1,
                                    place_prob=1)
    return text

In [5]:
for i in range(10):
    komo_text = komo_generator()
    print(komo_text)

komo = ry.KOMO(C, 4, 1, 2, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.position, ['omnibase'], ry.OT.sos, [1e0], [], 1)
komo.addObjective([0, 1], ry.FS.pose, ['stack_1_main_group_3_door_handle_handle_visual'], ry.OT.eq, [1e1], [], 1)
komo.addObjective([0, 1], ry.FS.qItself, ['stack_1_main_group_3_slidejoint'], ry.OT.eq, [1e1], [], 1)
komo.addObjective([2], ry.FS.qItself, ['stack_1_main_group_3_slidejoint'], ry.OT.eq, [1e1], [-1.033])
komo.addObjective([3], ry.FS.qItself, ['stack_1_main_group_3_slidejoint'], ry.OT.eq, [1e1], [1.033])
komo.addObjective([4], ry.FS.qItself, ['stack_1_main_group_3_slidejoint'], ry.OT.eq, [1e1], [-1.033])
komo.addObjective([1, 4], ry.FS.negDistance, ['l_gripper', 'stack_1_main_group_3_door_handle_handle_visual'], ry.OT.eq, [1e1], [0.079]

In [6]:
komo_samples = 10_000

outfolder = "./komo_samples"
text_outfolder = os.path.join(outfolder, "texts")
if not os.path.exists(text_outfolder):
    os.makedirs(text_outfolder)
else:
    for file in os.listdir(text_outfolder):
        file_path = os.path.join(text_outfolder, file)
        os.remove(file_path)

for i in tqdm(range(komo_samples)):
    komo_text = komo_generator()
    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "w") as file:
        file.write(komo_text)

100%|██████████| 10000/10000 [00:00<00:00, 39804.20it/s]


## Classify KOMO texts

In [7]:
total_komos = 0
total_feasible_komos = 0
total_error_komos = 0

feasible_komos = []
error_komos = []

for i in tqdm(range(komo_samples)):
    
    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "r") as file:
        komo_text = file.read()

    # print(f"Processing file {filename}...")
    total_komos += 1
    
    # try:
    komo = ry.KOMO(C, 1, 1, 0, False)
    exec(komo_text)
    ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
    # except:
    #     print(komo_text)
    #     exit()
    #     total_error_komos += 1
    #     error_komos.append(i)
    #     continue
    
    if ret.feasible:
        total_feasible_komos += 1
        feasible_komos.append(i)
    
    if (i+1)%(komo_samples//10) == 0:
        print(f"Total feasible komos: {total_feasible_komos}")
        print(f"Total error komos: {total_error_komos}")
        print(f"Total komos: {total_komos}")
        print(f"{(total_feasible_komos/total_komos)*100:.2f}% feasible")

 10%|▉         | 999/10000 [02:55<30:38,  4.90it/s]

Total feasible komos: 11
Total error komos: 0
Total komos: 1000
1.10% feasible


 20%|██        | 2001/10000 [05:49<18:39,  7.14it/s]

Total feasible komos: 16
Total error komos: 0
Total komos: 2000
0.80% feasible


 30%|███       | 3002/10000 [08:38<15:41,  7.43it/s]

...

Total feasible komos: 40
Total error komos: 0
Total komos: 4000
1.00% feasible


 50%|█████     | 5000/10000 [14:22<11:24,  7.30it/s]

Total feasible komos: 58
Total error komos: 0
Total komos: 5000
1.16% feasible


 58%|█████▊    | 5830/10000 [16:45<11:59,  5.80it/s]


KeyboardInterrupt: 

In [11]:
result = f"""Total feasible komos - Total error komos - Total komos: 
{total_feasible_komos} - {total_error_komos} - {total_komos}. 
({(total_feasible_komos/total_komos)*100:.2f}% feasible)"""
print(result)
print(feasible_komos)
results_file_path = os.path.join(outfolder, "feasibles.json")
results_dict = {
    "feasible_komos": feasible_komos,
    "error_komos": error_komos,
}
json.dump(results_dict, open(results_file_path, 'w'))

Total feasible komos - Total error komos - Total komos: 
67 - 0 - 5831. 
(1.15% feasible)
[4, 184, 196, 228, 334, 426, 504, 535, 595, 825, 878, 1132, 1199, 1287, 1343, 1505, 2207, 2417, 2441, 2485, 2526, 2551, 2666, 2721, 2735, 2822, 2876, 2881, 2929, 3014, 3080, 3121, 3266, 3351, 3353, 3421, 3713, 3827, 3917, 3969, 4180, 4272, 4372, 4381, 4411, 4450, 4461, 4532, 4644, 4727, 4738, 4793, 4862, 4872, 4909, 4967, 4977, 4994, 5076, 5103, 5293, 5316, 5326, 5381, 5429, 5665, 5819]


In [14]:
outfolder = "komo_samples_pivot_big_kitchen"
text_outfolder = os.path.join(outfolder, "texts")
feasible_komos = json.load(open(os.path.join(outfolder, "feasibles.json"), 'r'))["feasible_komos"]

## Solve submotions

In [ ]:
for i in feasible_komos:

    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "r") as file:
        komo_text = file.read()
    print(f"__________ SAMPLE {i} __________")
    print(komo_text)

    komo = ry.KOMO(C, 1, 1, 0, False)
    exec(komo_text)
    ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()

    komo.view(True)
    komo.view_play(True, delay=.5)

## Visualize feasible KOMOs

In [16]:
feasible_samples_count = 10
if len(feasible_komos) < feasible_samples_count:
    feasible_samples = feasible_komos.copy()
else:
    feasible_samples = np.random.choice(feasible_komos, feasible_samples_count, replace=False)

for i in feasible_samples:

    filename = os.path.join(text_outfolder, f"komo{i}.txt")
    with open(filename, "r") as file:
        komo_text = file.read()
    print(f"__________ SAMPLE {i} __________")
    print(komo_text)

    komo = ry.KOMO(C, 1, 1, 0, False)
    exec(komo_text)
    ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()

    komo.view(True)
    komo.view_play(True, delay=.5)

__________ SAMPLE 3351 __________
komo = ry.KOMO(C, 2, 1, 2, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 1, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.position, ['omnibase'], ry.OT.sos, [1e0], [], 1)
komo.addObjective([0, 1], ry.FS.pose, ['stack_3_main_group_1_door_handle_handle_visual'], ry.OT.eq, [1e1], [], 1)
komo.addObjective([0, 1], ry.FS.qItself, ['stack_3_main_group_1_doorhinge'], ry.OT.eq, [1e1], [], 1)
komo.addObjective([2], ry.FS.qItself, ['stack_3_main_group_1_doorhinge'], ry.OT.eq, [1e1], [0.265])
komo.addObjective([1, 2], ry.FS.positionDiff, ['r_gripper', 'stack_3_main_group_1_door_handle_handle_visual'], ry.OT.eq, [1e1])
komo.addObjective([1, 2], ry.FS.angularVel, ['r_gripper'], ry.OT.eq, [1e1])

__________ SAMPLE 2666 __________
komo = ry.KOMO(C, 2, 1, 2, True)
komo.addControlObjective([], 0, 1e